# **Cell 1: Import thư viện và thiết lập đường dẫn**

In [26]:
import pandas as pd
import os
import re

# Thư mục chứa file gốc
RAW_DATA_DIR = os.path.join("data", "raw-data")

# Thư mục lưu kết quả
PREPROCESSED_DIR = os.path.join("data", "preprocessed-data")

# Tạo thư mục preprocessed nếu chưa tồn tại
os.makedirs(PREPROCESSED_DIR, exist_ok=True)

# Kiểm tra thư mục raw data có tồn tại không
if not os.path.exists(RAW_DATA_DIR):
    raise FileNotFoundError(f"Không tìm thấy thư mục: {RAW_DATA_DIR}")

print(f"Đọc dữ liệu từ: {RAW_DATA_DIR}")
print(f"Lưu kết quả vào: {PREPROCESSED_DIR}")

Đọc dữ liệu từ: data\raw-data
Lưu kết quả vào: data\preprocessed-data


# **Cell 2: Lấy danh sách file và gán nhãn**

In [27]:
# Lấy danh sách file từ thư mục raw-data
files = [f for f in os.listdir(RAW_DATA_DIR) if f.lower().endswith(('.csv', '.xlsx'))]

if not files:
    print(f"Không tìm thấy file CSV/XLSX nào trong: {RAW_DATA_DIR}")
else:
    print(f"Tìm thấy {len(files)} file trong thư mục raw-data.")

# Gán nhãn dựa trên tên file
file_label_map = {}
for file in files:
    file_lower = file.lower()
    if "h-" in file_lower or "h for" in file_lower or "healthy" in file_lower:
        label = 0  # Healthy
    elif "erosion" in file_lower:
        label = 1  # Surface Erosion
    elif "crack" in file_lower:
        label = 2  # Cracked Blade
    elif "unbalance" in file_lower or "unbalnce" in file_lower or "imbalance" in file_lower:
        label = 3  # Mass Imbalance
    elif "twist" in file_lower or "twsist" in file_lower:
        label = 4  # Twist Blade Fault
    else:
        label = -1  # Không xác định
    file_label_map[file] = label

Tìm thấy 35 file trong thư mục raw-data.


# **Cell 3: Kiểm tra phân bố nhãn và file không xác định**

In [28]:
# Kiểm tra phân bố nhãn
label_counts = pd.Series(file_label_map.values()).value_counts()
label_names = ["Healthy", "Surface Erosion", "Cracked Blade", "Mass Imbalance", "Twist Blade Fault"]
print("Phân bố nhãn:")
for label in sorted(label_counts.keys()):
    if label == -1:
        print(f"Không xác định: {label_counts[label]} file")
    else:
        print(f"{label_names[label]}: {label_counts[label]} file")

# In danh sách file với label = -1
unknown_files = [file for file, label in file_label_map.items() if label == -1]
if unknown_files:
    print("\nCác file không xác định (label = -1):")
    for file in unknown_files:
        print(file)

Phân bố nhãn:
Healthy: 7 file
Surface Erosion: 7 file
Cracked Blade: 7 file
Mass Imbalance: 7 file
Twist Blade Fault: 7 file


# **Cell 4: Đọc và xử lý từng file (trích xuất dữ liệu, chuẩn hóa, gán nhãn và wind speed)**

In [29]:
# Đọc và gộp dữ liệu
data = []
for file, label in file_label_map.items():
    file_path = os.path.join(RAW_DATA_DIR, file)
    if not os.path.exists(file_path):
        print(f"File không tồn tại: {file_path}")
        continue
    print(f"Đang xử lý file: {file}")
    
    # Đọc file
    try:
        if file.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Không hỗ trợ định dạng file: {file}")
            continue
    except Exception as e:
        print(f"Lỗi khi đọc file {file}: {e}")
        continue
    
    # Chuẩn hóa dữ liệu
    columns = df.columns.tolist()
    print(f"Các cột trong file {file}: {columns}")  # Kiểm tra cột
    
    if "Time - Voltage_1" in df.columns and "Amplitude - Voltage_1" in df.columns:
        df = df.rename(columns={"Time - Voltage_1": "time", "Amplitude - Voltage_1": "amplitude"})
        df["amplitude"] = df["amplitude"].astype(float) * 10  # Chuyển từ Voltage sang g
    elif "Time - sec" in df.columns and "Amplitude - g" in df.columns:
        df = df.rename(columns={"Time - sec": "time", "Amplitude - g": "amplitude"})
        df["amplitude"] = df["amplitude"].astype(float)  # Đã ở đơn vị g
    elif "Time - Voltage_1;Amplitude - Voltage_1" in df.columns:
        time_amp = df["Time - Voltage_1;Amplitude - Voltage_1"].str.split(";", expand=True)
        time_amp = time_amp.dropna()
        time_amp = time_amp[time_amp[0].str.strip() != ""]
        time_amp = time_amp[time_amp[1].str.strip() != ""]
        if time_amp.empty:
            print(f"File {file} không có dữ liệu hợp lệ sau khi lọc.")
            continue
        try:
            df = df.loc[time_amp.index]
            df["time"] = time_amp[0].astype(float)
            df["amplitude"] = time_amp[1].astype(float) * 10
        except Exception as e:
            print(f"Lỗi khi chuẩn hóa dữ liệu trong file {file}: {e}")
            continue
    elif "Time - sec;Amplitude - g" in df.columns:
        time_amp = df["Time - sec;Amplitude - g"].str.split(";", expand=True)
        time_amp = time_amp.dropna()
        time_amp = time_amp[time_amp[0].str.strip() != ""]
        time_amp = time_amp[time_amp[1].str.strip() != ""]
        if time_amp.empty:
            print(f"File {file} không có dữ liệu hợp lệ sau khi lọc.")
            continue
        try:
            df = df.loc[time_amp.index]
            df["time"] = time_amp[0].astype(float)
            df["amplitude"] = time_amp[1].astype(float)
        except Exception as e:
            print(f"Lỗi khi chuẩn hóa dữ liệu trong file {file}: {e}")
            continue
    else:
        print(f"File {file} không có cột dữ liệu phù hợp (Time và Amplitude).")
        continue
    
    # Gán nhãn
    df["label"] = label
    
    # Trích xuất tốc độ gió – linh hoạt hơn
    match = re.search(r"(?:v ?wind|v ?w|vw|[._-]) ?=? ?([\d]+\.?\d*)", file.lower())
    if match:
        wind_speed_str = match.group(1)
        try:
            wind_speed = float(wind_speed_str)
            print(f"  → Tìm thấy wind_speed = {wind_speed} m/s\n")  # Debug
        except ValueError:
            print(f"Không thể chuyển đổi tốc độ gió từ file {file}: {wind_speed_str}")
            wind_speed = 0.0
    else:
        print(f"  Không tìm thấy tốc độ gió trong file {file}")
        wind_speed = 0.0
    df["wind_speed"] = wind_speed
    
    # Chỉ giữ các cột cần thiết
    df = df[["time", "amplitude", "label", "wind_speed"]]
    data.append(df)

Đang xử lý file: crack fault state-Vw=4.5.csv
Các cột trong file crack fault state-Vw=4.5.csv: ['Time - Voltage_1;Amplitude - Voltage_1']
  → Tìm thấy wind_speed = 4.5 m/s

Đang xử lý file: crack fault-Vw=4.csv
Các cột trong file crack fault-Vw=4.csv: ['Time - Voltage_1;Amplitude - Voltage_1']
  → Tìm thấy wind_speed = 4.0 m/s

Đang xử lý file: Crack Fault-Vw=5.4.csv
Các cột trong file Crack Fault-Vw=5.4.csv: ['Time - Voltage_1;Amplitude - Voltage_1']
  → Tìm thấy wind_speed = 5.4 m/s

Đang xử lý file: Crack Fault-Vw=5.csv
Các cột trong file Crack Fault-Vw=5.csv: ['Time - Voltage_1;Amplitude - Voltage_1']
  → Tìm thấy wind_speed = 5.0 m/s

Đang xử lý file: Crack State-Vw=1.3.csv
Các cột trong file Crack State-Vw=1.3.csv: ['Time - Voltage_1;Amplitude - Voltage_1']
  → Tìm thấy wind_speed = 1.3 m/s

Đang xử lý file: Crack state-Vw=2.8.csv
Các cột trong file Crack state-Vw=2.8.csv: ['Time - Voltage_1;Amplitude - Voltage_1']
  → Tìm thấy wind_speed = 2.8 m/s

Đang xử lý file: Crack state-V

# **Cell 5: Gộp dữ liệu, kiểm tra và lưu file**

In [30]:
if data:
    merged_data = pd.concat(data, ignore_index=True)
    
    print("\nThông tin dữ liệu gộp:")
    print(merged_data.info())
    print("\n5 dòng đầu:")
    print(merged_data.head())

    # Đường dẫn lưu
    output_path = os.path.join(PREPROCESSED_DIR, "merged_wind_turbine_data.csv")
    merged_data.to_csv(output_path, index=False)
    
    print(f"\nLưu File thành công:")
    print(f"   {output_path}")
    print(f"   Tổng cộng: {len(merged_data):,} mẫu")
else:
    print("Không có dữ liệu nào được xử lý!")


Thông tin dữ liệu gộp:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17500 entries, 0 to 17499
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   time        17500 non-null  float64
 1   amplitude   17500 non-null  float64
 2   label       17500 non-null  int64  
 3   wind_speed  17500 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 547.0 KB
None

5 dòng đầu:
    time  amplitude  label  wind_speed
0  0.000   0.000322      2         4.5
1  0.001  -0.013920      2         4.5
2  0.002   0.063230      2         4.5
3  0.003  -0.004760      2         4.5
4  0.004   0.051960      2         4.5

Lưu File thành công:
   data\preprocessed-data\merged_wind_turbine_data.csv
   Tổng cộng: 17,500 mẫu
